In [1]:
# Load env variables and create client
from dotenv import load_dotenv
from anthropic import Anthropic

load_dotenv()

client = Anthropic()
model = "claude-haiku-4-5"

In [2]:
# Helper functions
def add_user_message(messages, text):
    user_message = {"role": "user", "content": text}
    messages.append(user_message)


def add_assistant_message(messages, text):
    assistant_message = {"role": "assistant", "content": text}
    messages.append(assistant_message)


def chat(messages, system=None, temperature=1.0, stop_sequences=[]):
    params = {
        "model": model,
        "max_tokens": 1000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences,
    }

    if system:
        params["system"] = system

    message = client.messages.create(**params)
    return message.content[0].text

In [3]:
import json


def generate_dataset():
    prompt = """
Generate a evaluation dataset for a prompt evaluation. The dataset will be used to evaluate prompts
that generate Python, JSON, or Regex specifically for AWS-related tasks. Generate an array of JSON objects,
each representing task that requires Python, JSON, or a Regex to complete.

Example output:
```json
[
    {
        "task": "Description of task",
    },
    ...additional
]
```

* Focus on tasks that can be solved by writing a single Python function, a single JSON object, or a regular expression.
* Focus on tasks that do not require writing much code

Please generate 3 objects.
"""

    messages = []
    add_user_message(messages, prompt)
    add_assistant_message(messages, "```json")
    text = chat(messages, stop_sequences=["```"])
    return json.loads(text)

In [4]:
dataset = generate_dataset()

with open("dataset.json", "w") as f:
    json.dump(dataset, f, indent=2)

In [5]:
def run_prompt(test_case):
    """Merges the prompt and test case input, then returns the result"""
    prompt = f"""
    Please solve the following task:

    {test_case["task"]}
    """
    
    messages = []
    add_user_message(messages, prompt)
    output = chat(messages)
    return output

In [6]:
def run_test_case(test_case):
    """Calls run_prompt, then grades the result"""
    output = run_prompt(test_case)

    # TODO - Grading
    score = 10

    return {
        "output": output,
        "test_case": test_case,
        "score": score
    }

In [7]:
def run_eval(dataset):
    """Loads the dataset and calls run test case with each case"""
    results = []

    for test_case in dataset:
        result = run_test_case(test_case)
        results.append(result)

    return results

In [8]:
with open("dataset.json", "r") as f:
    dataset = json.load(f)

results = run_eval(dataset)

In [9]:
print(json.dumps(results, indent=2))

[
  {
    "output": "# AWS S3 Bucket ARN Region Extractor\n\nHere's a Python function that extracts the AWS region from an S3 bucket ARN:\n\n```python\ndef extract_region_from_s3_arn(arn: str) -> str:\n    \"\"\"\n    Extracts the AWS region from an S3 bucket ARN.\n    \n    Args:\n        arn (str): The S3 bucket ARN in format 'arn:aws:s3:region:account-id:bucket/bucket-name'\n    \n    Returns:\n        str: The AWS region (e.g., 'us-west-2')\n    \n    Raises:\n        ValueError: If the ARN format is invalid\n    \"\"\"\n    try:\n        parts = arn.split(':')\n        \n        # S3 ARN format: arn:aws:s3:region:account-id:bucket/bucket-name\n        if len(parts) < 4:\n            raise ValueError(\"Invalid ARN format\")\n        \n        region = parts[3]\n        \n        if not region:\n            raise ValueError(\"Region not found in ARN\")\n        \n        return region\n    \n    except (IndexError, AttributeError) as e:\n        raise ValueError(f\"Failed to parse A